In [1]:
import igraph as ig
import utils
import numpy as np
import itertools
from statistics import mean


In [2]:
def VISUAL_STYLE(g: ig.Graph) -> dict:
    return {
        'layout': utils.layout_geo(g),
        'bbox': (0, 0, 3000, 3000)
    }


In [3]:
g = ig.Graph.Read('../../data/graphs/april2022.graphml')


In [4]:
g_L_space = ig.Graph(directed=False)


In [5]:
# Adding all vertex from original graph
g_L_space.add_vertices(
    n=g.vs['name'],
    attributes={
        'label': g.vs['label'],
        'lat': g.vs['lat'],
        'lon': g.vs['lon']
    }
)


In [6]:
for a, b in itertools.combinations(g.vs, 2):
    edges = g.es.select(_between=([a.index], [b.index]))
    if len(edges) > 0:
        g_L_space.add_edge(
            source=a.index,
            target=b.index,
            time_second=mean([e['time_second'] for e in edges]),
            num_train=mean([e['num_train'] for e in edges])
        )


In [7]:
g_L_space.summary()


'IGRAPH UN-- 428 643 -- \n+ attr: label (v), lat (v), lon (v), name (v), num_train (e), time_second (e)'

In [8]:
betweeness = np.array(g_L_space.betweenness(directed=False))
norm_btwness = (betweeness - betweeness.min()) / (betweeness.max() - betweeness.min()) 

vs = {
    **VISUAL_STYLE(g_L_space),
    'vertex_size': 50*norm_btwness
}

print(f'Top 10 vertex by betweenness (normalized):')
for i in utils.top_n_indices(norm_btwness, 10):
    print(f"label: {g.vs[i]['label']}, betweeness: {norm_btwness[i]}")

ig.plot(g_L_space, **vs);

Top 10 vertex by betweenness (normalized):
label: MILANO ROGOREDO, betweeness: 1.0
label: MONZA, betweeness: 0.938707014853686
label: MILANO CENTRALE, betweeness: 0.9171965689535807
label: MILANO LAMBRATE, betweeness: 0.8798776392268836
label: TREVIGLIO, betweeness: 0.6895445879998946
label: LECCO, betweeness: 0.6760866978993241
label: MILANO PORTA GARIBALDI, betweeness: 0.5678815461221044
label: RHO FIERA, betweeness: 0.5545479541863134
label: MILANO BOVISA FNM, betweeness: 0.5408341260449161
label: BRESCIA, betweeness: 0.5357325359260636
